## Import

In [1]:
import sys
sys.path.append(r'C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\src\facility_location_Bergen\custome_modules')

In [2]:
import warnings
from shapely.errors import ShapelyDeprecationWarning
# Ignore the ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

In [3]:
import os
import dill
import random
import inspect
import numpy as np
import pandas as pd
import pickle as pkl
import geopandas as gpd
import pyomo.environ as pyo
from shapely.geometry import Point
from log import print_INFO_message_timestamp, print_INFO_message
from facility_location import AdjacencyMatrix, StochasticFacilityLocation
from retrieve_global_parameters import retrieve_adj_matrix_path

## Useful functions

In [4]:
def sample_idx(idxs, sample_ratio=0.1):
    n = len(idxs)
    s = random.sample(idxs, int(n*sample_ratio))
    return s

In [5]:
def sample_matrix(matrix, idx_sample):
    n_sample = len(idx_sample)
    sample_matrix = np.zeros((n_sample, n_sample))
    for i, r in enumerate(matrix[idx_sample, :]):
        sample_matrix[i,:] = r[idx_sample]
        
    return sample_matrix

In [6]:
def sample_coords(coordinates, idx_sample):
    sample_coords = coordinates.iloc[idx_sample]
    return sample_coords

In [7]:
times = ["morning", "midday", "afternoon"]

In [8]:
average_graphs = {}
time = "all_day"

print_INFO_message(f"Loading avg graph for {time}")
path = rf"C:\Users\Marco\Documents\GitHub\GeoSpatial-analysis\facility-location-Bergen\data\03_primary\average_graph_{time}.pkl"
with open(path, "rb") as f:
    average_graphs[time] = pkl.load(f)

                    INFO     Loading avg graph for all_day


In [9]:
adj_paths = {time: r"C:/Users/Marco/Documents/GitHub/GeoSpatial-analysis/facility-location-Bergen/" + retrieve_adj_matrix_path(time) for time in times}

In [10]:
adj_matricies = {time: None for time in times}

for time in times:
    with open(adj_paths[time], "rb") as f:
        adj_matricies[time] = pkl.load(f)

In [11]:
weighted_adj_matricies = {time: AdjacencyMatrix(adj_matrix=adj_matricies[time],
                                  kind="empirical",
                                  epsg=None,
                                  mode="time") for time in times}

## Problem initialization

It's not possible to solve the problem exactly using all the nodes in the graph. The problem is too big. We can try to solve it using a subset of the nodes.

random.seed(324324)

In [12]:
ratio1 = 1/50
ratio2= 1/20

In [13]:
coordinates = pd.Series(list(average_graphs["all_day"].nodes()))
coordinates = coordinates.apply(lambda x: Point(x))
coordinates = gpd.GeoDataFrame(geometry=coordinates)

In [14]:
idx_sampled = sample_idx(list(coordinates.index), ratio1)
idx_sampled2 = sample_idx(idx_sampled, ratio2)
coordinates_sampled = sample_coords(coordinates, idx_sampled)
coordinates_sampled2 = sample_coords(coordinates, idx_sampled2)

In [15]:
n_locations = 3

In [16]:
probabilities = {time: 1/len(weighted_adj_matricies) for time in times}

In [17]:
fl_stochastic = StochasticFacilityLocation(coordinates=coordinates_sampled,
                                           n_of_locations_to_choose=n_locations,
                                           candidate_coordinates=coordinates_sampled2,)

In [18]:
fl_stochastic.solve(scenarios_data=weighted_adj_matricies,
                    scenarioProbabilities=probabilities,
                    method="LS",
                    max_iter=10,)

                    INFO     Defining the abstract model...
[06/19/23 14:09:25] INFO     Initializing data...
[06/19/23 14:09:25] INFO     Solving the model...
[   98.36] Initializing SPBase


Current Iteration: 1 Time Elapsed:    0.00 Current Objective: -Inf
Current Iteration: 2 Time Elapsed:    0.16 Time Spent on Last Master:    0.02 Time Spent Generating Last Cut Set:    0.12 Current Objective: 1300.13
Current Iteration: 3 Time Elapsed:    0.29 Time Spent on Last Master:    0.05 Time Spent Generating Last Cut Set:    0.08 Current Objective: 1300.13
Current Iteration: 4 Time Elapsed:    0.44 Time Spent on Last Master:    0.04 Time Spent Generating Last Cut Set:    0.11 Current Objective: 1300.13
Converged in 4 iterations.
Total Time Elapsed:    0.69 Time Spent on Last Master:    0.20 Time spent verifying second stage:    0.05 Final Objective: 1370.40
morning x[305] 0.0
morning x[1868] 1.0
morning x[3462] 1.0
morning x[4047] 1.0
morning x[5021] 0.0
midday x[305] 0.0
midday x[1868] 1.0
midday x[3462] 1.0
midday x[4047] 1.0
midday x[5021] 0.0
afternoon x[305] 0.0
afternoon x[1868] 1.0
afternoon x[3462] 1.0
afternoon x[4047] 1.0
afternoon x[5021] 0.0


In [19]:
fl_stochastic.__dict__.keys()

dict_keys(['coordinates', 'candidate_coordinates', 'n_of_locations_to_choose', 'n_of_demand_points', 'model', 'scenarios_names', 'scenarios_probabilities', 'solution_value', 'locations_index', 'locations_coordinates', 'first_stage_solution', 'second_stage_solution', 'solver_status', 'computation_time'])

In [27]:
fl_stochastic.save("prova.pkl")

In [22]:
n_locations = 1

In [28]:
fl = StochasticFacilityLocation.load(rf"prova.pkl")